# Creating a Database of Scalar Values for WEST C5 Experimental Campaign 

In [1]:
# assume working in Jupyter Lab
%matplotlib inline 

%load_ext autoreload
%autoreload 2

In [2]:
# WEST libraries
import sys
if sys.platform == 'linux':
    sys.path.append('/Home/JH218595/pywed')
    sys.path.append('/Home/JH218595/IRFMtb')
    sys.path.append('/Home/JH218595/PPPAT')
else:
    sys.path.append('C:\\Users\\JH218595\\Documents\\pywed')
    sys.path.append('C:\\Users\\JH218595\\Documents\\IRFMtb')
    sys.path.append('C:\\Users\\JH218595\\Documents\\PPPAT')    

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
plt.rcParams['figure.figsize'] = (10,6)
try:
    from tqdm.notebook import tqdm
except ImportError as e:
    from tqdm import tqdm

try:
    from pppat.control_room.signals import *
except ImportError as e:
    sys.path.append('../../PPPAT/')
    from pppat.control_room.signals import *

from pulse_database import PulseDB

IMAS lib not available...


The database has been created in another notebook. Importing database : 

In [4]:
dt = 0.10 # s

hdf5_filename = 'databases/WEST_C5_pulse_data.hdf5'
file_output = 'WEST_C5_database_resumed_parameters.csv'

In [5]:
db = PulseDB(hdf5_filename)
print(f'Database contains {len(db.pulse_list)} shots, from #{db.pulse_list[0]} to #{db.pulse_list[-1]} ')

Database contains 1709 shots, from #56287 to #56927 


## Creating a meaningfull database with pandas
The idea is to split time in small pieces and to calculate scalar values for each of them

In [6]:
def split_in_pieces(y, t, nb_pieces):
    """Split a time signel y(t) into smaller piece of length dt, and return t, average, min, max and std of each of them"""
    y_mean_min_max, t_pieces = [], []
    if nb_pieces > 0: 
        ts = np.array_split(np.squeeze(t), nb_pieces)
        ys = np.array_split(np.squeeze(y), nb_pieces)
        for (_y, _t) in zip(ys, ts):
            # Get the mean, min and max values of the data in the time piece
            # Keep only if the mean value is within +/- 5% of the min/max
            # otherwise use NaN
            _mean, _mini, _maxi = mean_min_max(_y)
            
            if _mean == _mini == _maxi:  # no data
                y_mean_min_max.append([_mean, _mini, _maxi])           
                t_pieces.append(np.mean(_t))                
            elif (np.abs(_mean - _mini)/_mean < 30/100) and (np.abs(_mean - _maxi)/_mean < 30/100):
                y_mean_min_max.append([_mean, _mini, _maxi])           
                t_pieces.append(np.mean(_t))
            else:
                y_mean_min_max.append([np.nan, np.nan, np.nan]) 
                t_pieces.append(np.nan)
        return np.array(y_mean_min_max), np.array(t_pieces)    
    else:
        return np.array([np.nan, np.nan, np.nan]), np.array([np.nan])
    

In [7]:
data = pd.DataFrame()


for pulse in tqdm(db.pulse_list):
    # start as ip > 300 kA
    try:
        ip, t_ip = db.get_signal(pulse, 'Ip')

        t_start = t_ip[(ip > 0.3).squeeze()][0]
        t_end = t_ip[(ip > 0.3).squeeze()][-1]

        nb_pieces = int(np.round((t_end - t_start)/dt))

        rows = {'pulse': pulse}
        for signame in db.list_signal(pulse):           
            try:
                y, t = db.get_signal(pulse, signame)
                # Smooth some noisy signals
                if any(sig in signame for sig in ['Fe', 'Cu', 'Ag18', 'Ag19', 'Langmuir', 
                                                  'Prad', 'Prad_bulk', 'Prad_imas', 
                                                  'Rext', 'MHD',
                                                  'Rext',
                                                  'IC_Phase_Q1 (Pf_left - Pf_right)', 'IC_Phase_Q2 (Pf_left - Pf_right)', 'IC_Phase_Q4 (Pf_left - Pf_right)'
                                                  'IC_Phase_Q1', 'IC_Phase_Q2', 'IC_Phase_Q4',
                                                  'Rext_median', 'Rext_median_NICE',
                                                  'LH_P_tot', 'IC_P_tot', 'IC_P_Q1', 'IC_P_Q2', 'IC_P_Q4']):
                    y = smooth(y, window_length=21)

                # splitting signals in pieces
                _y, _t = in_between(y, t, t_start, t_end)
                ys, ts = split_in_pieces(_y, _t, nb_pieces)

                # Taking reference time from the plasma current
                if signame == 'Ip':
                    rows['time'] = np.squeeze(ts)

            except IndexError as e:
                # deals with resumed data (like IC frequencies):
                # replicate the data for the number of pieces
                if y.ndim == 1 and len(y)>1:
                    ys = np.tile(y, (int(nb_pieces),1))
                else:
                    # fill with 0 if empty arrays
                    ys = np.zeros(nb_pieces)

            except ValueError as e:
                ys = np.zeros(nb_pieces)

            # add data points into the Dataframe
            if signame == 'IC_Frequencies':
                try:
                    rows['freq_Q1'] = ys[:,0]
                    rows['freq_Q2'] = ys[:,1]
                    rows['freq_Q4'] = ys[:,2]
                except Exception as e:
                    rows['freq_Q1'] = np.nan
                    rows['freq_Q2'] = np.nan
                    rows['freq_Q4'] = np.nan                
            if signame == 'LH_Positions':
                rows['R_LH1'] = ys[:,0]
                rows['R_LH2'] = ys[:,1]
            if signame == 'IC_Positions':
                rows['R_Q1'] = ys[:,0]
                rows['R_Q2'] = ys[:,1]
                rows['R_Q4'] = ys[:,2]
            try:
                if signame == 'Datetime':
                    rows['year'] = ys[:,0]
                    rows['month']= ys[:,1]
                    rows['day']  = ys[:,2]
                    rows['hour'] = ts[:,0]
                    rows['minute'] = ts[:,1]
                    rows['second'] = ts[:,2]
            except Exception as e:
                pass
            else:

                if ys.ndim > 1:  # if to deal with the case of zeros (ValueError previously)
                    ys = ys[:,0]  # keep only mean (yet)
                rows[signame] = np.squeeze(ys)

            _df = pd.DataFrame(rows)
        # append data from each pulse to the final DF
        data = data.append(_df)
    except Exception as e:
        pass

/Home/JH218595/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Home/JH218595/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-6-ccc3c4664544>:16: RuntimeWarning: divide by zero encountered in double_scalars
  elif (np.abs(_mean - _mini)/_mean < 30/100) and (np.abs(_mean - _maxi)/_mean < 30/100):


Exporting the Dataframe:

In [8]:
data.columns

Index(['pulse', 'Ag18', 'Ag19', 'Cu', 'year', 'month', 'day', 'Fe', 'freq_Q1',
       'freq_Q2', 'freq_Q4', 'IC_Frequencies', 'IC_P_Q1', 'IC_P_Q2', 'IC_P_Q4',
       'IC_P_tot', 'IC_Phase_Q1', 'IC_Phase_Q1 (Pf_left - Pf_right)',
       'IC_Phase_Q2', 'IC_Phase_Q2 (Pf_left - Pf_right)', 'IC_Phase_Q4',
       'IC_Phase_Q4 (Pf_left - Pf_right)', 'R_Q1', 'R_Q2', 'R_Q4',
       'IC_Positions', 'IC_Rc_Q1_avg', 'IC_Rc_Q1_left', 'IC_Rc_Q1_right',
       'IC_Rc_Q2_avg', 'IC_Rc_Q2_left', 'IC_Rc_Q2_right', 'IC_Rc_Q4_avg',
       'IC_Rc_Q4_left', 'IC_Rc_Q4_right', 'IC_Voltage_left_lower_Q1',
       'IC_Voltage_left_lower_Q2', 'IC_Voltage_left_lower_Q4',
       'IC_Voltage_left_upper_Q1', 'IC_Voltage_left_upper_Q2',
       'IC_Voltage_left_upper_Q4', 'IC_Voltage_right_lower_Q1',
       'IC_Voltage_right_lower_Q2', 'IC_Voltage_right_lower_Q4',
       'IC_Voltage_right_upper_Q1', 'IC_Voltage_right_upper_Q2',
       'IC_Voltage_right_upper_Q4', 'time', 'Ip', 'Isotopic Ratio INBUM04',
       'Isotopic 

In [9]:
data.to_csv(file_output)

In [10]:
len(data)

102974

In [11]:
data.head()

,pulse,Ag18,Ag19,Cu,year,month,day,Fe,freq_Q1,freq_Q2,...,MHD,Ohmic_P,Prad,Prad_imas,Rext_median,Rext_median_NICE,Separatrix_P,W_MHD,frad_imas,nl
0,56289,1.330797,0.922018,NaN,2020.0,12.0,10.0,NaN,55.299999,55.799999,...,0.0,0.0,0.0,0.090342,2981.637954,2973.267766,0.0,41387.948887,NaN,1.449843
1,56289,0.983774,1.000000,NaN,2020.0,12.0,10.0,NaN,55.299999,55.799999,...,0.0,0.0,0.0,0.089766,2946.637476,2968.167753,0.0,47250.362452,NaN,1.562838
2,56289,1.000000,1.000000,NaN,2020.0,12.0,10.0,6.086065,55.299999,55.799999,...,0.0,0.0,0.0,0.088568,2956.318470,2951.450858,0.0,48850.911413,NaN,1.579303
3,56289,1.000000,NaN,1.896461,2020.0,12.0,10.0,NaN,55.299999,55.799999,...,0.0,0.0,0.0,0.095068,2942.756561,2946.898475,0.0,49309.269950,39.939243,1.579081
4,56289,1.000000,NaN,NaN,2020.0,12.0,10.0,3.738001,55.299999,55.799999,...,0.0,0.0,0.0,0.106382,2950.297941,2946.527634,0.0,50541.130810,37.665285,1.586296
